# Using pre-trained OCP models as ASE calculators

For those interested in using our pretrained models for other applications, we provide an [ASE](https://wiki.fysik.dtu.dk/ase/#:~:text=The%20Atomic%20Simulation%20Environment%20(ASE,under%20the%20GNU%20LGPL%20license.)-compatible Calculator to interface with ASE's functionality.

## Download pretrained checkpoint

We have released checkpoints of all the models on the leaderboard [here](https://github.com/Open-Catalyst-Project/ocp/blob/master/MODELS.md). These trained models can be used as an ASE calculator for various calculations.

For this tutorial we download our current best model checkpoint: GemNet-T

In [1]:
!wget -q -nc https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_08/s2ef/gemnet_t_direct_h512_all.pt
checkpoint_path = "gemnet_t_direct_h512_all.pt"

## Using the OCP Calculator


In [ ]:
import os

import ase.io
import numpy as np
from ase.build import add_adsorbate, fcc100, molecule
from ase.constraints import FixAtoms
from ase.optimize import BFGS
from ocpmodels.common.relaxation.ase_utils import OCPCalculator

# Construct a sample structure
adslab = fcc100("Cu", size=(3, 3, 3))
adsorbate = molecule("C3H8")
add_adsorbate(adslab, adsorbate, 3, offset=(1, 1))
tags = np.zeros(len(adslab))
tags[18:27] = 1
tags[27:] = 2
adslab.set_tags(tags)
cons = FixAtoms(indices=[atom.index for atom in adslab if (atom.tag == 0)])
adslab.set_constraint(cons)
adslab.center(vacuum=13.0, axis=2)
adslab.set_pbc(True)

# Define the calculator
calc = OCPCalculator(checkpoint=checkpoint_path)

# Set up the calculator
adslab.calc = calc

os.makedirs("data/sample_ml_relax", exist_ok=True)
opt = BFGS(adslab, trajectory="data/sample_ml_relax/toy_c3h8_relax.traj")

opt.run(fmax=0.05, steps=100)

fatal: detected dubious ownership in repository at '/opt/ocp'
To add an exception for this directory, call:

	git config --global --add safe.directory /opt/ocp


amp: false
cmd:
  checkpoint_dir: /home/jovyan/ml_catalysis_tutorials/ml-catalysis-tutorials/notes/checkpoints/2022-10-29-20-24-32
  commit: null
  identifier: ''
  logs_dir: /home/jovyan/ml_catalysis_tutorials/ml-catalysis-tutorials/notes/logs/tensorboard/2022-10-29-20-24-32
  print_every: 100
  results_dir: /home/jovyan/ml_catalysis_tutorials/ml-catalysis-tutorials/notes/results/2022-10-29-20-24-32
  seed: null
  timestamp_id: 2022-10-29-20-24-32
dataset: null
gpus: 0
logger: tensorboard
model: gemnet_t
model_attributes:
  activation: silu
  cbf:
    name: spherical_harmonics
  cutoff: 6.0
  direct_forces: true
  emb_size_atom: 512
  emb_size_bil_trip: 64
  emb_size_cbf: 16
  emb_size_edge: 512
  emb_size_rbf: 16
  emb_size_trip: 64
  envelope:
    exponent: 5
    name: polynomial
  extensive: true
  max_neighbors: 50
  num_after_skip: 2
  num_atom: 3
  num_before_skip: 1
  num_blocks: 3
  num_concat: 1
  num_radial: 128
  num_spherical: 7
  otf_graph: true
  output_init: HeOrthogona

In [ ]:
import matplotlib.pyplot as plt
from ase.visualize.plot import animate

# the `index` argument corresponds to what frame of the trajectory to read in, specifiying ":" reads in the full trajectory.
traj = ase.io.read("data/sample_ml_relax/toy_c3h8_relax.traj", index=":")
anim = animate(traj, radii=0.8, rotation=("-75x, 45y, 10z"))

rc("animation", html="jshtml")
anim

In [ ]:
import pandas as pd
import plotly.express as px

energies = [
    image.get_potential_energy()
    for image in traj
]

df = pd.DataFrame({"Relaxation Step": range(len(energies)), "GemNet-T Adsorption Energy [eV]": energies})
fig = px.line(df, x="Relaxation Step", y="GemNet-T Adsorption Energy [eV]")
fig.show()